In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_xml(r'path-to-mismatch-file.xml',xpath='interval/edge')

In [ ]:
df

In [ ]:
def GEH(m, c):
    return np.sqrt((2 * (m - c)**2)/(m+c))

In [ ]:
df['outputCount'] = df['measuredCount'] - df['deficit']

In [ ]:
df = df.sort_values(by = 'deficit', ascending = False)

In [ ]:
df.head(50) # inspect worst mismatches

In [ ]:
df['GEH'] = df.apply(lambda x: GEH(x['outputCount'], x['measuredCount']), axis = 1) # calculate GEH

In [ ]:
df

In [ ]:
df[df['GEH']<5] # filter on GEH < 5

In [ ]:
df = df.sort_values(by = 'GEH', ascending = False)

In [ ]:
plt.bar(list(range(4214)), df.reset_index()['GEH']) # show descending GEH

In [ ]:
# GEH Histogram
fig, ax = plt.subplots()
ax.hist(df['GEH'])
ax.set_ylabel('Absolute frequency')
ax.set_xlabel('GEH in √(vehicles / h)')

In [ ]:
# 5 < GEH <= 10
df[(df['GEH']>5)&(df['GEH']<=10)]

In [ ]:
len(df[(df['GEH']>10)])

In [ ]:
201/4214

In [ ]:
31/4214

## Affinity index

In [ ]:
def affinity(x, y):
    base = x if x > y else y
    counter = x if x < y else y
    return np.abs(counter / base)

In [ ]:
df['AffinityIndex'] = df.apply(lambda x: affinity(x['outputCount'], x['measuredCount']), axis = 1)

In [ ]:
df

In [ ]:
df = df.sort_values(by = 'AffinityIndex', ascending = False)

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "font.size":28
})

In [ ]:
x = list(range(len(df)))
x = [(i/(len(x)-1)) * 100 for i in x]

In [ ]:
import matplotlib.ticker as ticker
fig, ax = plt.subplots(figsize = (9, 9))
ax.bar(x=x, height = df['AffinityIndex']*100, width = 1.0)
ax.set_ylabel('Affinity in \%')
ax.set_xlabel('Edges in \%')
ax.xaxis.set_major_locator(ticker.MultipleLocator(10))
ax.yaxis.set_major_locator(ticker.MultipleLocator(10))
ax.yaxis.grid(color = 'lightgray')
ax.xaxis.grid(color = 'lightgray')
fig.tight_layout()
fig.savefig('affinityindex.pdf')

In [ ]:
## Measured vs. modeled count

In [ ]:
fig, ax = plt.subplots(figsize = (9, 9))
ax.scatter(df['measuredCount'], df['outputCount'])
ax.set_xlabel('Input count')
ax.set_ylabel('Modeled count')
ax.xaxis.set_major_locator(ticker.MultipleLocator(500))
fig.tight_layout()
fig.savefig('countComparisonScatter.pdf')